# Assignment 2 (Stage Classification of Liver Cancer)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.preprocessing import Normalizer
import numpy as np
import matplotlib.pyplot as plt 

## To read Train and Test datasets

In [2]:
data_train=pd.read_csv('train.csv')
data_test=pd.read_csv("test.csv")

# data=pd.concat(data)
# print(data_train.head(5))
# print(data_test.head(5))
print (data_train.shape)
print (data_test.shape)
data_train_lable=data_train['Label']
data_train.drop(['Label','id'],axis=1,inplace=True)
data_test_ID=data_test['id']
data_test.drop(['id'],axis=1,inplace=True)

# data_train,data_train_lable=shuffle(data_train,data_train_lable)
# print (data_train.columns)
# print(data_test.columns)
print((data_train.shape),data_train_lable.shape,(data_test.shape))

# data=pd.concat([data_train,data_test])
# print(data.shape)
print(data_test_ID.shape)


(279, 60485)
(71, 60484)
(279, 60483) (279,) (71, 60483)
(71,)


### To normalize the datasets

In [3]:
transformer = Normalizer().fit(data_train)
data_train=pd.DataFrame(transformer.transform(data_train))
transformer = Normalizer().fit(data_test)
data_test=pd.DataFrame(transformer.transform(data_test))
# print("done")
print(data_train.head(5))
data_train1=data_train
data_test1=data_test


      0         1         2      3         4         5      6         7      \
0  0.003546  0.000000  0.006989    0.0  0.003784  0.008776    0.0  0.000017   
1  0.000000  0.000019  0.004417    0.0  0.001766  0.007063    0.0  0.000019   
2  0.000000  0.000000  0.003586    0.0  0.003812  0.006606    0.0  0.000015   
3  0.000548  0.000000  0.003956    0.0  0.004264  0.007748    0.0  0.000017   
4  0.000110  0.000000  0.005758    0.0  0.003637  0.006998    0.0  0.000063   

      8         9      ...  60473     60474  60475     60476     60477  \
0  0.018661  0.000710  ...    0.0  0.007268    0.0  0.001168  0.000000   
1  0.015817  0.000000  ...    0.0  0.002398    0.0  0.000615  0.000434   
2  0.015581  0.000280  ...    0.0  0.018540    0.0  0.000477  0.000493   
3  0.016852  0.000107  ...    0.0  0.013844    0.0  0.000185  0.000000   
4  0.020690  0.000000  ...    0.0  0.000361    0.0  0.000947  0.000000   

      60478     60479  60480     60481  60482  
0  0.010470  0.000047    0.0  0.

In [4]:
# X_train=data_train1
# X_test=data_test1

### pre feature selection using ExtraTreesClassifier it reduces features to 1000

In [5]:
from sklearn.ensemble import ExtraTreesClassifier 
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5, criterion ='entropy', max_features = 1000)
extra_tree_forest.fit(data_train1, data_train_lable) 
feature_importance = extra_tree_forest.feature_importances_ 

feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_], 
                                        axis = 0)
print(sum(feature_importance_normalized))
# for i in range(8):
#     print(feature_importance_normalized[i])
i=0
dictn={}
# print(len(X_train.columns))
for t in data_train1.columns:
#     print("yes")
    if(feature_importance_normalized[i]>0):
#         print(feature_importance_normalized[i])
        dictn[t]=feature_importance_normalized[i]
        
    i=i+1
# print(len(dictn))
dictn=sorted(dictn.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)
# print(len(dictn))
# print(dictn[0:100])
# plt.bar(data_train.columns, feature_importance_normalized) 
# plt.xlabel('Feature Labels') 
# plt.ylabel('Feature Importances') 
# plt.title('Comparison of different Feature Importances') 
# plt.show()

# print(data_train.head(5))

2.0


In [6]:
imp_feature=[]
for x in dictn:
#     print(x[0])
    imp_feature.append(x[0])
# print(imp_feature)

data_train2=data_train1[imp_feature]
data_test2=data_test1[imp_feature]

## Feature selection using SelectKBest ( It reduces features from 1000 to 100)

In [7]:
# pca = PCA(n_components=100)
# data = pca.fit_transform(data)
# X_train, X_test, y_train, y_test = train_test_split(data_train, data_train_lable, test_size=0.2)
X_train=data_train2
y_train=data_train_lable
X_test=data_test2
# --------------------------------------------------------------------------
select_feature = SelectKBest(chi2, k=100).fit(X_train, y_train)

selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)
X_train_chi = pd.DataFrame(select_feature.transform(X_train))

X_test_chi = pd.DataFrame(select_feature.transform(X_test))
print(X_train_chi.columns,X_test_chi.columns)
# -------------------------------------------------------------------------------
# print(X_test.head(5))

RangeIndex(start=0, stop=100, step=1) RangeIndex(start=0, stop=100, step=1)


# GridSearchCV for find a better param grid for different models

In [10]:
from sklearn.ensemble import RandomForestClassifier
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    kernels=['linear','rbf']
    param_grid_svm = {'C': Cs, 'gamma' : gammas,'kernel':kernels}
#     param_grid_randomforest=   {'bootstrap': [True, False],'max_depth': [7,10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],'max_features': ['auto', 'sqrt'],'min_samples_leaf': [1, 2, 4],'min_samples_split': [2, 5, 10]}
#     grid_search = GridSearchCV(svm.SVC(), param_grid, cv=nfolds)
#     param_grid_mlpClassifier = {'solver': ['lbfgs'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
    param_grid_randomforest=   {'bootstrap': [True, False],'max_depth': [7,10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],'max_features': ['auto', 'sqrt'],'min_samples_leaf': [1, 2, 4],'min_samples_split': [2, 5, 10],'random_state':[0,1,2]}

    grid_search = GridSearchCV(RandomForestClassifier(), param_grid_randomforest, cv=nfolds,scoring = 'roc_auc')

    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_,grid_search
params,clf=svc_param_selection(X_train_chi, y_train, 5)
print(params)
print("",clf.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

...

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

{'bootstrap': False, 'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'random_state': 2}
 0.7240006859800457


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python

### Because of random_state accuracy is changing so i capture the params and give to the model

In [32]:
# from sklearn.neural_network import MLPClassifier
# clf = MLPClassifier(activation='relu',solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100, 50), random_state=40)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(bootstrap= False, max_depth= 7, max_features= 'auto', min_samples_leaf= 4, min_samples_split= 10, random_state= 1)
# clf = RandomForestClassifier(bootstrap= False, max_depth= 7, max_features='auto', min_samples_leaf= 1, min_samples_split= 5, random_state= 2)

clf.fit(X_train_chi,y_train)



/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=7, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

## Prediction for the test data and store it in csv

In [33]:

predictions=clf.predict(X_test_chi)
# df_merged = pd.merge(data_test_ID, pd.DataFrame(predictions ))
# df_merged['ID'] =data_test_ID
# data_test_ID=data_test_IDpredictions
# print(data_test_ID.head())
print(predictions.shape,data_test_ID.shape)
prediction = pd.DataFrame({'Id':data_test_ID,'Label': predictions})
# print(prediction)
# pd.to_csv('check.csv')
prediction.to_csv (r'output_dataframe_RandomForest.csv', index = False, header=True)

(71,) (71,)
